# Opencv之人脸仿射
## 储黄瑞

## 该代码实现利用人脸的五点仿射变换实现人脸对齐
## 具体就是首先使用mtcnn检测算法检测出人脸区域，并得到lanmarks关键点坐标和检测框坐标之后对人脸区域外扩60%，然后对该外扩后的区域重新得到关键点，进行五点仿射变换得到即可。
## 参考链接：https://blog.csdn.net/oTengYue/article/details/79278572
## 如果图片中检测到多张人脸，或者未检测到人脸，那我就把原图复制过去。

In [12]:
#基本的引入，按字母表排序
import cv2
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
from numpy import random
from numpy.random import uniform
import os
from os import path as osp
import PIL
from PIL import Image, ImageDraw
import random
from random import shuffle
import sys
sys.path.append(r'py')
import time
#torch部分的import
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
#from tensorboardX import SummaryWriter
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from MTCNN_detector import MtcnnDetector,vis_face

# MTCNN  model

In [4]:
class P_Net(nn.Module):
    def __init__(self):
        super(P_Net, self).__init__()
        self.pre_layer = nn.Sequential(
            # 12x12x3
            nn.Conv2d(3, 10, kernel_size=3, stride=1),  # conv1
            nn.PReLU(),  # PReLU1
            # 10x10x10
            nn.MaxPool2d(kernel_size=2, stride=2),  # pool1
            # 5x5x10
            nn.Conv2d(10, 16, kernel_size=3, stride=1),  # conv2
            # 3x3x16
            nn.PReLU(),  # PReLU2
            nn.Conv2d(16, 32, kernel_size=3, stride=1),  # conv3
            # 1x1x32
            nn.PReLU()  # PReLU3
        )
        # detection
        self.conv4_1 = nn.Conv2d(32, 1, kernel_size=1, stride=1)
        # bounding box regresion
        self.conv4_2 = nn.Conv2d(32, 4, kernel_size=1, stride=1)
        # landmark localization
        self.conv4_3 = nn.Conv2d(32, 10, kernel_size=1, stride=1)
        # weight initiation with xavier
        self.apply(weights_init)

    def forward(self, x):
        x = self.pre_layer(x)
        det = torch.sigmoid(self.conv4_1(x))
        box = self.conv4_2(x)
        #landmark = self.conv4_3(x)
        # det:[,2,1,1], box:[,4,1,1], landmark:[,10,1,1]
        return det, box#, landmark


In [5]:
class R_Net(nn.Module):
    def __init__(self):
        super(R_Net, self).__init__()
        self.pre_layer = nn.Sequential(
            # 24x24x3
            nn.Conv2d(3, 28, kernel_size=3, stride=1),  # conv1
            nn.PReLU(),  # prelu1
            # 22x22x28
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool1
            # 10x10x28
            nn.Conv2d(28, 48, kernel_size=3, stride=1),  # conv2
            nn.PReLU(),  # prelu2
            # 8x8x48
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool2
            # 3x3x48
            nn.Conv2d(48, 64, kernel_size=2, stride=1),  # conv3
            # 2x2x64
            nn.PReLU()  # prelu3
        )
        # 2x2x64
        self.conv4 = nn.Linear(64 * 2 * 2, 128)  # conv4
        # 128
        self.prelu4 = nn.PReLU()  # prelu4
        # detection
        self.conv5_1 = nn.Linear(128, 1)
        # bounding box regression
        self.conv5_2 = nn.Linear(128, 4)
        # lanbmark localization
        self.conv5_3 = nn.Linear(128, 10)
        # weight initiation weih xavier
        self.apply(weights_init)

    def forward(self, x):
        x = self.pre_layer(x)
        x = x.view(x.size(0), -1)
        x = self.conv4(x)
        x = self.prelu4(x)
        det = torch.sigmoid(self.conv5_1(x))
        #det = F.softmax(self.conv5_1(x), dim=1)#也可以试试softmax呀
        box = self.conv5_2(x)
        #landmark = self.conv5_3(x)
        return det, box#, landmark

In [6]:
class O_Net(nn.Module):
    def __init__(self):
        super(O_Net, self).__init__()
        self.pre_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1),  # conv1
            nn.PReLU(),  # prelu1
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool1
            nn.Conv2d(32, 64, kernel_size=3, stride=1),  # conv2
            nn.PReLU(),  # prelu2
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool2
            nn.Conv2d(64, 64, kernel_size=3, stride=1),  # conv3
            nn.PReLU(),  # prelu3
            nn.MaxPool2d(kernel_size=2, stride=2),  # pool3
            nn.Conv2d(64, 128, kernel_size=2, stride=1),  # conv4
            nn.PReLU()  # prelu4
        )
        self.conv5 = nn.Linear(128 * 2 * 2, 256)  # conv5
        self.prelu5 = nn.PReLU()  # prelu5
        # detection
        self.conv6_1 = nn.Linear(256, 1)
        # bounding box regression
        self.conv6_2 = nn.Linear(256, 4)
        # lanbmark localization
        self.conv6_3 = nn.Linear(256, 10)
        # weight initiation weih xavier
        self.apply(weights_init)

    def forward(self, x):
        x = self.pre_layer(x)
        x = x.view(x.size(0), -1)
        x = self.conv5(x)
        x = self.prelu5(x)
        # detection
        det = torch.sigmoid(self.conv6_1(x))
        box = self.conv6_2(x)
        landmark = self.conv6_3(x)
        return det, box, landmark


## 相关的一些functions

In [7]:
 # 1. 根据网络层的不同定义不同的初始化方式     
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        #nn.init.constant_(m.bias, 0) bias不要全初始化为0
        nn.init.normal_(m.bias, mean=0, std=1)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
     # 是否为批归一化层
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
#使用这样的初始化后，模型的初始表现确实好了些

In [8]:
def load_net(save_folder, net_name):
    # pnet, rnet, onet = P_Net(), R_Net(), O_Net()
    net_list = {'pnet': P_Net(), 'rnet': R_Net(), 'onet': O_Net()}

    try:
        net = net_list[net_name].to(device)
        try:
            print('===> loading the saved net weights...')
            _ = osp.join(save_folder, net_name + '.pkl')
            print('===> check {} saved path({}):{}'.format(net_name, _, osp.exists(_)))
            net.load_state_dict(torch.load(_, map_location=device))
            return net  # , rnet, onet
        except Exception:
            print('*** fail to load the saved net weights!')
            return net
    except Exception:
        print('*** Net name wrong!')

# 利用OpenCV实现仿射人脸

In [9]:
# 最终的人脸对齐图像尺寸分为112x112，并分别对应结果图像中的两组仿射变换目标点,如下所示
imgSize2 = [112,112]
coord5point2 = [[30.2946+8.0000, 51.6963], # 112x112的目标点
               [65.5318+8.0000, 51.6963],
               [48.0252+8.0000, 71.7366],
               [33.5493+8.0000, 92.3655],
               [62.7299+8.0000, 92.3655]]

def transformation_from_points(points1, points2):
    '''0 - 先确定是float数据类型 '''
    points1 = points1.astype(np.float64)
    points2 = points2.astype(np.float64)
    '''1 - 消除平移的影响 '''
    c1 = np.mean(points1, axis=0)
    c2 = np.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2
    '''2 - 消除缩放的影响 '''
    s1 = np.std(points1)
    s2 = np.std(points2)
    points1 /= s1
    points2 /= s2
    '''3 - 计算矩阵M=BA^T；对矩阵M进行SVD分解；计算得到R '''
    U, S, Vt = np.linalg.svd(points1.T * points2)
    R = np.dot(U, Vt)
    '''4 - 构建仿射变换矩阵 '''
    s = s2/s1
    sR = s*R
    c1 = c1.reshape(2,1)
    c2 = c2.reshape(2,1)
    T = c2 - np.dot(sR,c1) # 模板人脸的中心位置减去 需要对齐的中心位置（经过旋转和缩放之后）
    trans_mat = np.hstack([sR,T])   # 2x3
    return trans_mat

def warp_im(img_im, orgi_landmarks,tar_landmarks):
    pts1 = np.float64(np.matrix([[point[0], point[1]] for point in orgi_landmarks]))
    pts2 = np.float64(np.matrix([[point[0], point[1]] for point in tar_landmarks]))
    M = transformation_from_points(pts1, pts2)
    dst = cv2.warpAffine(img_im, M[:2], (img_im.shape[1], img_im.shape[0]),borderMode=cv2.BORDER_TRANSPARENT)
    return dst

In [72]:
#为处理lfw数据集改编版，输出是142*142,在datapreprocess的时候可以resize一下。
def opencv_image(pic_path):
    # 对一个路径下的所有图片进行对齐，并保存在new_picture_path下
    new_pic_path="Cropped_images"
    #加载mtcnn参数
    save_folder="CHUHR"
    pnet= load_net(save_folder,'pnet')
    rnet= load_net(save_folder,'rnet')
    onet= load_net(save_folder,'onet')
    mtcnn_detector = MtcnnDetector(pnet=pnet,rnet=rnet,onet=onet,min_face_size=12,threshold=[0.1,0.5,0.9])
    count=0
    people_name_list = os.listdir(pic_path)
    people_name_list.sort()
    for people_name in people_name_list:
        pic_name_list=os.listdir(os.path.join(pic_path,people_name))
        for every_pic_name in pic_name_list:
            image_path=os.path.join(pic_path,people_name,every_pic_name)
            img_im = cv2.imread(image_path)       
            if img_im is None:
                continue
            else:
                img = Image.open(image_path)
                #关键点检测
                bounding_boxes, points= mtcnn_detector.detect_face(img)
                shape = img_im.shape
                height = shape[0]
                width = shape[1]
                # 处理该张图片中的每个框
                if bounding_boxes.shape[0] !=1:
                    img_save_path=os.path.join(new_pic_path ,people_name)
                    cv2.imwrite(img_save_path+"/"+every_pic_name[:-4]+'.jpg',img_im)
                else:
                    for i in range(bounding_boxes.shape[0]):  # 根据行号得到每张图片有多少个回归框
                        x1, y1, x2, y2 = int(bounding_boxes[i][0]), \
                                         int(bounding_boxes[i][1]), \
                                         int(bounding_boxes[i][2]), \
                                         int(bounding_boxes[i][3])
                        # 外扩大60%，防止对齐后人脸出现过多黑边
                        new_x1 = max(int(1.30 * x1 - 0.30 * x2),0)
                        new_x2 = min(int(1.30 * x2 - 0.30 * x1),width-1)
                        new_y1 = max(int(1.30 * y1 - 0.30 * y2),0)
                        new_y2 = min(int(1.30 * y2 - 0.30 * y1),height-1)


                        landmarks_one = points[i, :]
                        landmarks_one = landmarks_one.reshape((5, 2))

                        # 得到原始图中关键点坐标
                        left_eye_x = landmarks_one[0][0]
                        right_eye_x = landmarks_one[1][0]
                        nose_x = landmarks_one[2][0]
                        left_mouth_x = landmarks_one[3][0]
                        right_mouth_x = landmarks_one[4][0]
                        left_eye_y = landmarks_one[1][1]
                        right_eye_y = landmarks_one[0][1]
                        nose_y = landmarks_one[2][1]
                        left_mouth_y = landmarks_one[3][1]
                        right_mouth_y = landmarks_one[4][1]

                        # 得到外扩60%后图中关键点坐标
                        new_left_eye_x = left_eye_x - new_x1
                        new_right_eye_x = right_eye_x - new_x1
                        new_nose_x = nose_x - new_x1
                        new_left_mouth_x = left_mouth_x - new_x1
                        new_right_mouth_x = right_mouth_x - new_x1
                        new_left_eye_y = left_eye_y - new_y1
                        new_right_eye_y = right_eye_y - new_y1
                        new_nose_y = nose_y - new_y1
                        new_left_mouth_y = left_mouth_y - new_y1
                        new_right_mouth_y = right_mouth_y - new_y1

                        face_landmarks = [[new_left_eye_x,new_left_eye_y], # 在扩大60%人脸图中关键点坐标
                                          [new_right_eye_x,new_right_eye_y],
                                          [new_nose_x,new_nose_y],
                                          [new_left_mouth_x,new_left_mouth_y],
                                          [new_right_mouth_x,new_right_mouth_y]]
                        face = img_im[new_y1: new_y2, new_x1: new_x2] # 扩大60%的人脸区域
                        dst2 = warp_im(face,face_landmarks,coord5point2) # 112x112对齐后尺寸
                        crop_im2 = dst2[0:imgSize2[0]+30,0:imgSize2[1]+30]#最后扩大取值区域，防止截取的人脸不完整。
                        img_save_path=os.path.join(new_pic_path ,people_name)
                        if not os.path.exists(img_save_path):
                            os.makedirs(img_save_path)
                        cv2.imwrite(img_save_path+"/"+every_pic_name[:-4]+'.jpg',crop_im2)

In [58]:
p1 = os.listdir("Cropped_images")

In [59]:
len(p1)

5749